In [18]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using Distributions
using JLD2

In [19]:
id = "1stm"
@load "assets/input/jld2/single_subunits/$(id)_protor.jld2" template_mol template_radii x_init
n_mol = 5
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n_mol]...);
mkpath("assets/input/jld2/$(n_mol)_$(id)_init/")


"assets/input/jld2/5_1stm_init"

In [10]:
bounds = 120.0
for i in 1:50
    x_init = vcat([
        [rand(Uniform(0.0, 2*pi)), rand(Uniform(0.0, 2*pi)), rand(Uniform(0.0, 2*pi)), 
        rand(Uniform(0.0, bounds)), rand(Uniform(0.0, bounds)), rand(Uniform(0.0, bounds))] 
        for i in 1:n_mol]...);
        
    @save "assets/input/jld2/$(n_mol)_$(id)_init/$(id)_protor_$(i).jld2" template_mol template_radii x_init
end

In [20]:

hotstart_folder = "assets/hpc_out_d/5_1stm_oj_0_0_os_0_85/"
for file in readdir(hotstart_folder)
    if split(file, ".")[end] == "jld2"
        @load "$hotstart_folder$file" in_out_data
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[3])
        template_mol = in_out_data.input.template_mol
        template_radii = in_out_data.input.template_radii
        @save "assets/input/jld2/5_1stm_hotstart/1stm_protor_$(id).jld2" template_mol template_radii state
    end
end